In [1]:
! pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 735.8 kB/s eta 0:00:00


In [2]:
import openai
import json
import pandas as pd
import time
from tqdm.auto import tqdm

openai.api_key = 'your_open_ai_key'

In [3]:
train = pd.read_csv('/content/textgraphs_konstruktor_pred.csv')

In [25]:
test_part = pd.DataFrame(columns = ['question', 'question_id', 'answers'])

In [26]:
ques_ids = train.question_id.unique()

In [27]:
for i in tqdm(range(1000)):
  cur_ques_id = ques_ids[i]
  test_part.loc[i, 'question_id'] = cur_ques_id
  test_part.loc[i, 'question'] = train.query('question_id == @cur_ques_id').question.unique()[0]
  test_part.loc[i, 'answers'] = ', '.join(train.query('question_id == @cur_ques_id').answerEntity.unique())

  0%|          | 0/1000 [00:00<?, ?it/s]

In [28]:
test_part['question_id'] = test_part['question_id'].astype("string")

In [29]:
test_part

,question,question_id,answers
0,"After publishing A Time to Kill, which book di...",0.0,Kongenes kamp
1,"Among the European Union countries, which one ...",1.0,"Crown of the Kingdom of Poland, German Reich, ..."
2,"Among the Final Fantasy games, which installme...",2.0,"Final Fantasy IV, Final Fantasy VII"
3,At which Academy Awards was the Leonardo DiCap...,3.0,"Academy Award for Best Actor, Academy Award fo..."
4,"Between Ulysses S Grant and Abe Lincoln, which...",4.0,"Abraham Lincoln, John Quincy Adams, Theodore R..."
...,...,...,...
995,Who's the president of Bolivia?,995.0,Luis Arce
996,"Who's won more NBA titles, Knicks or Celtics?",996.0,"Boston Celtics, Boston Celtics all-time roster..."
997,Whom did Elizabeth Bennet's older sister from ...,997.0,"Anthony Trollope, Charles Bingley, Edmund Mort..."
998,With which book did P. H. Newby win the Booker...,998.0,"All the King's Men, Little Lord Fauntleroy, Se..."


In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for j, row in tqdm(test_part.iterrows(), total=len(test_part)):
    response = openai.ChatCompletion.create(
        model="gpt-4-0125-preview",

        messages=[
                              {
            "role": "system",
            "content": "Generate the answer using chain-of-thought reasoning.\
            After generation verify that your answer is in the set of possible answers given\
            in the question. If your answer is not there, select the answer from the set.\
             At the end write the short answer after the phrase 'So the answer is:'"},
            {"role": "user",
            "content":
            "Q: "+row['question']+" Possible answers: "+row['answers']}
    ],
    )
    answer = response['choices'][-1]['message']['content']

    with open('/content/drive/MyDrive/Text2Graph/text2graph_test_with_konstruktor.jsonl', 'a+') as f:
        f.write(json.dumps({'id': row['question_id'], 'predText': answer, 'question': row['question']}) + '\n')


  0%|          | 0/1000 [00:00<?, ?it/s]

In [46]:
pred_gpt = pd.read_json(path_or_buf='/content/drive/MyDrive/Text2Graph/text2graph_test_with_konstruktor.jsonl', lines=True)

In [40]:
pred_gpt.loc[2, 'predText']

"To determine which Final Fantasy game had the highest worldwide sales between Final Fantasy IV and Final Fantasy VII, let's consider their impacts and popularities. Final Fantasy IV was originally released for the SNES in 1991 and was quite popular, introducing a more dynamic and involving story compared to its predecessors. It helped solidify the series' popularity but was still primarily successful within Japan and among a growing audience in the West.\n\nFinal Fantasy VII, on the other hand, was released for the PlayStation in 1997 and marked a significant turning point not just for the series but for the entire RPG genre. It was the first game in the series to feature 3D graphics and cinematics, which, combined with a deep, engaging storyline and memorable characters like Cloud Strife and Sephiroth, catapulted it to unprecedented worldwide acclaim and sales. Final Fantasy VII is widely regarded not just as a high point of the series, but as one of the greatest video games of all t

In [47]:
len(pred_gpt)

1000